# Подготовка данных и обучение

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np

### Использовать буду только самые необходимые признаки.
1. Оценка разговора + флаг использования продукта из таблицы communications
2. Прохождение курсов пользователями из таблицы courses_passing

In [2]:
# Загружаем два датасета
communications = pd.read_csv(
    "data/communications.csv", \
        sep=';', dtype={'employee_id': 'category'}, parse_dates=['communication_dt'])
courses_passing = pd.read_csv(
    "data/courses_passing.csv", \
        sep=';', dtype={'employee_id': 'category'}, parse_dates=['start_dt', 'end_dt'])

### Идея заключается в следующем.
1. Отберём тех пользователей, которые курс окончили.
2. Зададим промежуток времени до курса и после курса.
3. Создадим взвешенную меру эффективности курса.
4. Разделим датасет по мере эффективности каждого курса <br>для каждого человека на "до" и "после"
5. Посчитаем разницу
6. Составим матрицу оценок
7. Обучим SVD

In [3]:
# Предлагаю считать только тех, кто курсы окончил
courses_passed = courses_passing[~courses_passing['end_dt'].isnull()]
cols_to_include = ['start_dt', 'course_id', 'employee_id', 'end_dt']
courses_passed = courses_passed.loc[:, cols_to_include]

In [4]:
# Создаём промежуток размером в месяц до начала курса
courses_passed['month_before'] = courses_passed['start_dt'] \
    - pd.DateOffset(days=30)
# Создаём промежуток размером в месяц после окончания курса
courses_passed['month_after'] = courses_passed['end_dt'] \
    + pd.DateOffset(days=30)

In [5]:
# Удаляем неинформативный атрибут
communications = communications.drop('communication_id', axis=1)

In [6]:
# Предлагаю считать оценку разговора по следующей формуле
communications['weighted_mark'] = \
    communications['util_flg'] * 100 * 0.5 \
    + communications['communication_score'] * 0.5
# Так мы учтём влияние прихода клиента в общей оценке,
# как немаловажный показатель эффективности труда.

In [7]:
# Объединяем две таблицы. Получаем для каждой оценки интервалы
# которые сможем использовать как "ДО" и "ПОСЛЕ"
merged = communications \
    .merge(courses_passed, how='inner', on='employee_id')

In [8]:
# Удалим ненужное
del communications, courses_passed

In [9]:
# Удалим более ненужные признаки
merged = merged \
    .drop(['communication_score', 'util_flg'], axis=1)
# Просмотр первых нескольких строк объединенной таблицы
merged.head()

,communication_dt,employee_id,weighted_mark,start_dt,course_id,end_dt,month_before,month_after
0,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-06-25,79,2023-07-05,2023-05-26,2023-08-04
1,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-07-06,74,2023-07-15,2023-06-06,2023-08-14
2,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-06-03,16,2023-06-09,2023-05-04,2023-07-09
3,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-07-13,8,2023-07-20,2023-06-13,2023-08-19
4,2023-02-07,269d837a-fada-308d-d4ae-ab28ca2d57e4,42.5,2023-04-09,42,2023-04-15,2023-03-10,2023-05-15


In [10]:
# Коммуникации, которые были после курса и в пределах месяца
# следующего как контрольный после конца курса.
is_after = (merged['communication_dt'] > merged['end_dt'])\
    & (merged['communication_dt'] < merged['month_after'])
    
# Коммуникации, которые были до курса и в пределах месяца
# следующего как контрольный до начала курса.
is_before = (merged['communication_dt'] < merged['start_dt']) \
    & (merged['communication_dt'] > merged['month_before'])

In [11]:
# Считаем среднюю оценку до курса и после
mark_after_course = merged[is_after] \
    .groupby(['employee_id', 'course_id'])['weighted_mark'] \
        .agg('mean')
mark_before_course = merged[is_before] \
    .groupby(['employee_id', 'course_id'])['weighted_mark'] \
        .agg('mean')

In [12]:
# Считаем разницу
diff_df = (mark_after_course - mark_before_course)
diff_df = diff_df.unstack()
# Смотрим что получилось
diff_df.head()

course_id,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
employee_id,,,,,,,,,,,,,,,,,,,,,
00411460-f7c9-2d21-24a6-7ea0f4cb5f85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.593618,-0.60589,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006f52e9-102a-8d3b-e2fe-5614f42ba989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.588911,NaN,NaN,NaN,NaN
00a03ec6-533c-a7f5-c644-d198d815329c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-5.523111,NaN,NaN,NaN,NaN,NaN,7.075201,NaN,NaN
00ac8ed3-b432-7bdd-4ebb-ebcb2ba10a00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00e26af6-ac3b-1c1c-49d7-c3d79c60d000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.128442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Небольшой subtotal.
1. Мы вычислили средние взвешенные оценки разговоров пользователей.
2. Посчитали разницу во взвешенных оценках в промежутке "до" и "после"

Теперь будем приводить к единому масштабу.

In [13]:
def scale_row(row):
    min_val = row.min()
    max_val = row.max()
    if pd.isna(min_val) or pd.isna(max_val) or min_val == max_val:
        return row
    else:
        return (row - min_val) / (max_val - min_val) * 9 + 1


# Применяем функцию к каждой строке
diff_df = diff_df.apply(scale_row, axis=1)

diff_df = diff_df.fillna(0)

In [14]:
diff_df.iloc[0, :].min()

0.0

In [15]:
# удалим лишнее из памяти
del merged

In [16]:
# Выполнение SVD разложения
U, sigma, Vt = np.linalg.svd(diff_df, full_matrices=False)

# Восстановление оценок
reconstructed_data = np.dot(U[:, :len(sigma)] * sigma, Vt)
reconstructed_data

array([[-2.82191214e-16,  7.12342463e-16,  3.90789266e-15, ...,
        -1.09997792e-15,  3.08067165e-15,  4.22613723e-15],
       [ 1.53976973e-15,  5.91474004e-15, -2.70513027e-15, ...,
        -3.65419843e-15,  6.96291395e-15, -7.36315288e-15],
       [-4.95420108e-15, -5.22600210e-15, -3.57512888e-15, ...,
         7.36230940e+00,  2.81300244e-15, -3.19413268e-15],
       ...,
       [-2.51334263e-15,  6.64569893e-15, -5.04041366e-15, ...,
         2.65953792e-15,  1.75578740e-17,  5.83010114e-15],
       [ 1.16813338e-16, -1.28810483e-15,  3.60825193e-16, ...,
         4.69463394e+00,  4.67574125e-15,  1.45903078e-15],
       [ 1.99729998e-15,  1.12278766e-15,  3.66431004e-15, ...,
        -1.28184750e-15,  5.51799596e-15, -2.85427817e-15]])

In [17]:
svd_matrix = pd.DataFrame(data=reconstructed_data,
             index=diff_df.index, columns=diff_df.columns)

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Вычисление среднеквадратичной ошибки
mse = mean_squared_error(diff_df, reconstructed_data)
print("Mean Squared Error (MSE):", mse)

# Вычисление средней абсолютной ошибки
mae = mean_absolute_error(diff_df, reconstructed_data)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 2.453384142948036e-29
Mean Absolute Error (MAE): 2.4874131864290465e-15


#### Обе метрики близки к нулю, поэтому модель довольно хорошо реконструировала исходные данные, а значит может быть использована в качестве рекомендаций

In [19]:
# Рекомендации топ рекомендаций для первого пользователя
svd_matrix.iloc[0, :].argsort().sort_values(ascending=False)

course_id
75    91
69    90
22    89
9     88
24    87
      ..
71     4
53     3
72     2
37     1
28     0
Name: 00411460-f7c9-2d21-24a6-7ea0f4cb5f85, Length: 92, dtype: int64

### Интерпретация
Для пользователя под id "00411460-f7c9-2d21-24a6-7ea0f4cb5f85" наиболее рекомендуемый курс под id 75, на втором месте под id 69 и так далее